In [2]:
#dependencies
import numpy as np
import cv2
import os
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import keras


ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.multiarray failed to import

In [23]:
#preprocessing
def read_images(path):
    images = []
    all_paths = os.listdir(path)
    mini_set = all_paths[:400]
    for i in mini_set:
        file = path+"/"+i
        image = cv2.imread(file)
        image = cv2.resize(image,(64,64))
        images.append(image)

    return images

def rgb_to_lab(images):
    lab_images = []
    for i in images:
        lab_image= cv2.cvtColor(i, cv2.COLOR_BGR2LAB)
        lab_images.append(lab_image)

    return lab_images

def create_train_data(l,a,b):
    train_data = []
    for i in l:
        train_data.append(np.array(i.flatten(),dtype= 'float32'))
    train_labels_a = []
    train_labels_b = []
    for i in a:
        train_labels_a.append(np.array(i.flatten(),dtype='float32'))
    for i in b:
        train_labels_b.append(np.array(i.flatten(),dtype='float32'))

    return train_data, train_labels_a, train_labels_b

def extract_channels(lab_images):
    l_channels = []
    a_channels = []
    b_channels = []
    for i in lab_images:
        l,a,b = cv2.split(i)
        l_channels.append(l)
        a_channels.append(a)
        b_channels.append(b)

    return np.array(l_channels), np.array(a_channels), np.array(b_channels)

In [29]:
#helper
path = "C:/Users/Arghyadeep/Desktop/image colorization/new process/val2017"
images = read_images(path)
lab_images = rgb_to_lab(images)
l,a,b = extract_channels(lab_images)
tr, lbla, lblb = create_train_data(l,a,b)
train_size = 350
train_data = np.array(tr[:train_size])
eval_data = np.array(tr[train_size:])
train_labels_a = np.array(lbla[:train_size])
train_labels_b = np.array(lblb[:train_size])
train_labels = []
for i in range(train_size):
    temp = np.concatenate((train_labels_a[i],train_labels_b[i]),axis=0)
    train_labels.append(temp)
train_labels = np.array(train_labels,dtype='float32')
print(train_labels.shape)

eval_labels_a = np.array(lbla[train_size:])
eval_labels_b = np.array(lblb[train_size:])
eval_labels = []
for i in range(50):
    temp = np.concatenate((eval_labels_a[i],eval_labels_b[i]),axis=0)
    eval_labels.append(temp)
eval_labels = np.array(eval_labels)
print(eval_labels.shape)

(350, 8192)
(50, 8192)


In [30]:
#tf.logging.set_verbosity(tf.logging.INFO)



def cnn_model(features, labels, mode):
    #input layer
    input_layer = tf.reshape(features["x"],[-1,64,64,1])

    #convolution and pool layer 1
    conv1 = tf.layers.conv2d(inputs=input_layer,
                             filters=32,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)


    #convolution and pool layer 2
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=64,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    #convolution and pool layer 3
    conv3 = tf.layers.conv2d(inputs=pool2,
                             filters=128,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)


    #convolution and pool layer 4
    conv4 = tf.layers.conv2d(inputs=input_layer,
                             filters=256,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)
    
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    flat = tf.contrib.layers.flatten(inputs = pool4)

    fc1 = tf.layers.dense(inputs = flat,units = 1024)

    dropped = tf.layers.dropout(fc1, rate=0.2,
                                training = mode == tf.estimator.ModeKeys.TRAIN)

    fc2 = tf.layers.dense(inputs = dropped, units = 2*64*64)

    

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = fc2)

    loss = tf.losses.mean_squared_error(labels = labels, predictions = fc2)


    #logging_hook = tf.train.LoggingTensorHook({"loss" : loss}, every_n_iter=1)

    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,
                                      global_step = tf.train.get_global_step()
                                      )
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=loss,
                                          train_op = train_op)
    
    eval_metric_ops = {"accuracy":tf.metrics.accuracy(labels=labels,
                                                      predictions = fc2)
        }

    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss = loss,
                                      eval_metric_ops = eval_metric_ops)



In [49]:
model = tf.estimator.Estimator(model_fn = cnn_model)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x":train_data},
                                                        y = train_labels,
                                                        batch_size = 10,
                                                        num_epochs = None,
                                                        shuffle = True)
    
    
model.train(input_fn = train_input_fn,
                steps = 201)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":eval_data},
                                                       y = eval_labels,
                                                       num_epochs=1,
                                                       shuffle=False)

eval_results = model.evaluate(input_fn = eval_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\ARGHYA~1\\AppData\\Local\\Temp\\tmp2gbhedwe', '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ARGHYA~1\AppData\Local\Temp\tmp2gbhedwe\model.ckpt.
INFO:tensorflow:loss = 17932.047, step = 1


KeyboardInterrupt: 

In [59]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":train_data[:2]},
                                                       shuffle=False)
x = list(model.predict(input_fn = predict_input_fn))

INFO:tensorflow:Restoring parameters from C:\Users\ARGHYA~1\AppData\Local\Temp\tmp2gbhedwe\model.ckpt-1


In [62]:
a = x[1].reshape(8192,1)


In [64]:
a1 = a[:64*64]
a2 = a[64*64:]
a0 = train_data[1]

In [67]:
a0 = a0.reshape(64,64)
a1 = a1.reshape(64,64)
a2 = a2.reshape(64,64)

In [68]:
print(a0.shape,a1.shape,a2.shape)

(64, 64) (64, 64) (64, 64)


In [70]:
x = cv2.merge((a0,a1,a2))

In [77]:
cv2.imshow('img',x)

In [3]:
cv2.imshow('image',a0)

NameError: name 'a0' is not defined